### Import packages

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import splrep, BSpline

### Set Path

In [2]:
cd '/home/kaixin/Downloads/dircap_new'


/home/kaixin/Downloads/dircap_new


In [3]:
if (os.path.exists('output')==False):
    os.mkdir('output')
run_dircap=True
print_wavefunc=True
Start_E=0.028
Truncate_E=1.508
if print_wavefunc:
    outputfolder='./output_hs_wf/'
else:
    outputfolder='./output_hs/'

### Define Input Variables

In [4]:
Title="23Na+p:1st"
AT=22.989
ZT=11
AP=1.007276
ZP=1
Q=11.69269 # MeV
Ex=11.390 # MeV
WS_r=1.25			# WS radius parameter ***  1.25
WS_a=0.65 		    # WS diffuseness *** 0.65
# HS_r=1.25			# HS radius parameter ***  1.25
HS_r=1.25             # For hard sphere scattering potential
JP=0.5             # JP
JT=1.5             # JT
JF=1             # JF ***
LF=1             # LF ***
NR=2.0		    	# NR Number of nodes, including origin ***
LI=1.0             # LI 
Mult="E1"              # Multipolarity (E1, E2, M1)
Nstep=5000			# No. of steps 
StepSize=0.1             # Step size (fm)
if print_wavefunc:
    StepPrint=1.0             # if nonzero print wavefunctions at these steps
else:
    StepPrint=0             # if nonzero print wavefunctions at these steps
E_unit=0               # (0,1) for energies in (cm,lab) 
read_E=0               # read energies from external file (no=0,yes=1)
E_start=0.008  		    # Start energy (MeV)
E_step=0.01			# Energy step size (MeV)
E_number=250             # Number of energies 

WriteList=(Title,AT,ZT,AP,ZP,Q,Ex,WS_r,WS_a,HS_r,JP,JT,JF,LF,NR,LI,Mult,Nstep,StepSize,StepPrint,E_unit,read_E,E_start,E_step,E_number)
formatting=''
for i in range(0,len(WriteList)):
    formatting=formatting+'%s\n'

### Update inputs from external file and run multiple times

In [5]:
input_list=pd.read_csv('dc_list',delim_whitespace=True)
outfile_names=[]
for i in range(0,len(input_list)):
    outfile_name=''
    for v in input_list.columns:
        globals()[v]=input_list[v][i]  # find the defined variable from the input variable list to be updated 
        outfile_name=outfile_name+'%s'%globals()[v]+'_' # define output filename for this run
    outfile_name = outfile_name[:-1]
    outfile_names.append(outfile_name)
    WriteList=(Title,AT,ZT,AP,ZP,Q,Ex,WS_r,WS_a,HS_r,JP,JT,JF,LF,NR,LI,Mult,Nstep,StepSize,StepPrint,E_unit,read_E,E_start,E_step,E_number)
    if (run_dircap):
        print('Ex=',Ex,' NR=',NR,' LI=',LI)
        with open("dircap.in","w+") as outfile: # create input file for dircap code
            outfile.write(formatting % WriteList) 
        os.system('./dircap') # run dircap
#         os.system('cp '+'dircap.out '+'./output/'+outfile_name)
        os.system('cp '+'dircap.out '+outputfolder+outfile_name)
os.system('cp '+'dc_list '+outputfolder)
os.system('cp '+'levels '+outputfolder)


Ex= 0.0  NR= 1  LI= 1

 Direct Capture for Woods-Saxon potential. Version 4.0

 Careful with angular momentum coupling! Code
 returns finite values for non-physical
 combinations of ell_i and ell_f


Ex= 0.0  NR= 1  LI= 3

 Direct Capture for Woods-Saxon potential. Version 4.0

 Careful with angular momentum coupling! Code
 returns finite values for non-physical
 combinations of ell_i and ell_f




/tmp/ipykernel_346218/3206090568.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  input_list=pd.read_csv('dc_list',delim_whitespace=True)
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


Ex= 1.369  NR= 2  LI= 1

 Direct Capture for Woods-Saxon potential. Version 4.0

 Careful with angular momentum coupling! Code
 returns finite values for non-physical
 combinations of ell_i and ell_f


Ex= 1.369  NR= 1  LI= 1

 Direct Capture for Woods-Saxon potential. Version 4.0

 Careful with angular momentum coupling! Code
 returns finite values for non-physical
 combinations of ell_i and ell_f


Ex= 1.369  NR= 1  LI= 3

 Direct Capture for Woods-Saxon potential. Version 4.0

 Careful with angular momentum coupling! Code
 returns finite values for non-physical
 combinations of ell_i and ell_f


Ex= 4.123  NR= 1  LI= 1

 Direct Capture for Woods-Saxon potential. Version 4.0

 Careful with angular momentum coupling! Code
 returns finite values for non-physical
 combinations of ell_i and ell_f


Ex= 4.123  NR= 1  LI= 3

 Direct Capture for Woods-Saxon potential. Version 4.0

 Careful with angular momentum coupling! Code
 returns finite values for non-physical
 combinations of ell_i an

0

### Read output files

In [6]:
levels=pd.read_csv('levels',delim_whitespace=True)
Ex_index=input_list.columns.tolist().index('Ex')
LF_index=input_list.columns.tolist().index('LF')
LI_index=input_list.columns.tolist().index('LI')
# summed_S_coef=np.array([0,0,0]) # summed S(0),S'(0),S''(0) coefficients multiplied by C2S for each run.
summed_S_coef=np.arange(Start_E,Truncate_E,0.01)*0
listofsummed_S_coef=[]
listoflevel=[]
listofSvalue=[]
summed_S=None
with open("./output_hs/S_coefficients","w+") as outfile:
    outfile.write('\t'.join(str(value) for value in ['filename','1/2S\'\'(0)','S\'(0)','S(0)','Error']))
    outfile.write('\n')
    print('level','maximum of S(E)','C2S')
    for filename in outfile_names:
#         if float(filename[0:5])<7: # filter based on energy
#             continue
        data=pd.read_csv('./output_hs/'+filename,skiprows=24,delim_whitespace=True,names=["E","Sigma","S","S/b^2",'Phase'])
        E=data["E"].to_numpy().tolist()
        E=E[E.index(Start_E):E.index(Truncate_E)]
        S=data["S"].to_numpy().tolist()[0:len(E)] 
        data=levels.loc[(levels['Level'] == 1000*float(filename.split('_')[Ex_index])) \
                        & (levels['lf'] == float(filename.split('_')[LF_index]))\
                        ]
        for i in range (0,len(S)):
            S[i]=S[i]*data['C2S_MCMC'].values[0]
#             S[i]=S[i]*1000 # Convert the unit from Mev.barn to keV.barn! 

        if summed_S is None:
            summed_S=np.array(S)
        else:
            summed_S=summed_S+np.array(S)
#         Using 2nd order polynomial to fit. 
#         trend = np.polyfit(E,S,2)
#         trendpoly = np.poly1d(trend)  
#         Using smoothing interpolation. 
        tck = splrep(E, S, s=len(E))
        # Search for C2S by level energy and lf. Note that the unit should be converted from MeV.b 
        # to keV.b for RattesMC code, multiply by 1000.
#         data=levels.loc[(levels['Level'] == float(filename.split('_')[Ex_index])) & (levels['lf'] == float(filename.split('_')[LF_index]))]
#         summed_S=summed_S+data['(2J+1)C2S*2/3_MCMC'].values*trend

#  Old dircap code
#         summed_S_coef=summed_S_coef+trendpoly
#         listofsummed_S_coef.append(trendpoly)
        xrange=np.arange(Start_E, Truncate_E, 0.01)
        SmoothFit=BSpline(*tck)(xrange)
        summed_S_coef=summed_S_coef+SmoothFit
        listofsummed_S_coef.append(SmoothFit)
        
        listoflevel.append(str(data['Level'].values[0])+'keV')
        listofSvalue.append(max(S))
        outfile.write(filename)
        outfile.write('\t')
#         outfile.writelines('\t'.join(str(value) for value in list(trend)))
#         outfile.write('\t')
#         outfile.write(str(max(abs(trendpoly(E)-S)/S)))
#         outfile.write('\n')
        print(filename[0:6],max(S),data['C2S_MCMC'].values[0])

level maximum of S(E) C2S
0.0_2_ 0.00018540774 0.0574
0.0_2_ 2.3447326e-06 0.0574
1.369_ 0.000136752 0.0066
1.369_ 0.00081684708 0.2681
1.369_ 1.14328564e-05 0.2681
4.123_ 5.4993899999999995e-05 0.0118
4.123_ 1.0019734e-06 0.0118
4.238_ 0.0003803282 0.0197
4.238_ 0.00024353207999999998 0.0951
4.238_ 4.496613300000001e-06 0.0951
5.235_ 0.00018423616 0.0556
5.235_ 3.859696399999999e-06 0.0556
6.011_ 1.426212e-05 0.0036
6.011_ 3.34836e-07 0.0036
7.349_ 8.180820000000001e-05 0.0047
7.349_ 4.12434e-06 0.0022
7.349_ 1.231054e-07 0.0022
7.555_ 1.35945e-05 0.0009
7.555_ 1.0700099999999998e-06 0.0009
7.747_ 0.00034255760000000005 0.0332
7.747_ 4.8752759999999996e-05 0.0457
7.747_ 1.5851044999999999e-06 0.0457
8.358_ 0.00027752399999999997 0.009
8.358_ 2.4878699999999996e-05 0.009
8.437_ 0.00018258785999999998 0.0639
8.437_ 7.027083e-06 0.0639
8.437_ 0.0007176775000000001 0.0551
8.437_ 6.526595e-05 0.0551
8.654_ 0.0003937467 0.0233
8.654_ 3.467198e-05 0.0227
8.654_ 1.4161849e-06 0.0227


/tmp/ipykernel_346218/764051158.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  levels=pd.read_csv('levels',delim_whitespace=True)
/tmp/ipykernel_346218/764051158.py:18: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data=pd.read_csv('./output_hs/'+filename,skiprows=24,delim_whitespace=True,names=["E","Sigma","S","S/b^2",'Phase'])
/tmp/ipykernel_346218/764051158.py:18: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data=pd.read_csv('./output_hs/'+filename,skiprows=24,delim_whitespace=True,names=["E","Sigma","S","S/b^2",'Phase'])
/tmp/ipykernel_346218/764051158.py:18: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+

8.864_ 0.0013209424 0.0661
8.864_ 0.00013050784 0.0661


FileNotFoundError: [Errno 2] No such file or directory: './output_hs/9.003_2_2_0_1_E1'

In [ ]:
filename

In [ ]:
Label_selected = [x for _,x in sorted(zip(listofSvalue,listoflevel))][-10:]
Label_selected.reverse()
# Label_selected=Label_selected[0:]
print(Label_selected)
S_coef_selected = [x for _,x in sorted(zip(listofSvalue,listofsummed_S_coef))][-10:]
# S_coef_selected = S_coef_selected[0:]
S_coef_selected.reverse()
# print(S_coef_selected)
# Label_selected[0]='10731keV'
for i in range(0,len(Label_selected)):
#     S_coef_selected[i]=np.array([0,0,0])
    S_coef_selected[i]=np.arange(Start_E, Truncate_E, 0.01)*0
    for j in range(0,len(listoflevel)):
        if (Label_selected[i]==listoflevel[j]):
#             print(listoflevel[j],S_coef_selected[i],listofsummed_S_coef[j])
            S_coef_selected[i]=S_coef_selected[i]+listofsummed_S_coef[j]

In [ ]:
levels.loc[(levels['Level'] == float(filename.split('_')[Ex_index])) \
                        & (levels['lf'] == float(filename.split('_')[LF_index]))\
                        ]

In [ ]:
print(E[40],summed_S[40])

In [ ]:
# plt.scatter(E,summed_S,marker='.',color='red')
# trendpoly = np.poly1d(summed_S_coef)  
# plt.plot(E,trendpoly(E),color='b')
# plt.show()
# plt.scatter(E,abs(trendpoly(E)-summed_S)/summed_S,color='b',marker='.')
# plt.show()

In [ ]:
# ###### trendpoly = np.poly1d(summed_S_coef)  
# plt.title('My DC S-factor')
# plt.scatter(E,summed_S,marker='.',color='red',label='S-factor')
trend = np.polyfit(E,summed_S,2)
trendpoly = np.poly1d(trend)  
print(trend[2]*1000,trend[1],trend[0]*2/1000)
Es=np.array([0,8,18,28,38,48,98,148,198,298,398,498,598,798,998,1498,1998,2498])
formatted_str = '\t'.join(f'{x:.2f}' for x in Es)
print(Es)
formatted_str = '\t'.join(f'{x:.2f}' for x in 1000*trendpoly(Es/1000))
print(formatted_str)
plt.plot(E,trendpoly(E),color='b',label='fit-quad')

# plt.plot(E,summed_S_coef,color='b',label='fit-quad')
plt.xlabel('Energy/MeV')
plt.ylabel('S/MeV')
plt.legend()
plt.show()
# plt.scatter(E,abs(trendpoly(E)-summed_S)/summed_S,color='b',marker='.')
# plt.show()
# plt.savefig('MySfactor_10059-11518.png')
# plt.show()

# trendpoly2= np.poly1d([1/2*6.42e-3, -7.31e-3, 2.48e-2]) # Coefficients from RatesMC input,
#                                                  #converted to MeV barn.
# # trendpoly2= np.poly1d([0.00257,-0.00693,0.0239]) # From Hale.(2004)
# plt.scatter(E,trendpoly2(E),marker='.',color='orange')
# plt.title('DC S-factor from RatesMC input (Hale04)')
# # plt.savefig('RatesMC.png')
# plt.show()
# plt.scatter(E,trendpoly(E)/trendpoly2(E),marker='.')
# plt.title('Ratio')
# plt.show()
fig, ax = plt.subplots(figsize=(8, 6))
# plt.title('Fractional contributions to the total DC S-factor (ZS)',fontsize=22)
plt.title('HS',fontsize=25)
plt.xlabel('$E_{c.m.}$/MeV',fontsize=25)
plt.ylabel('Fraction',fontsize=25)
# plt.xticks(np.arange(0.08,0.25,0.04),fontsize=20)
# plt.yticks(np.arange(0.02,0.17,0.02),fontsize=20)
# plt.ylim(0.017,0.15)
# Eend=17  //format to compare with notre dame paper
plt.xticks(np.arange(0.1,0.81,0.1),fontsize=20)
plt.yticks(np.arange(0.02,0.20,0.02),fontsize=20)
plt.ylim(0.017,0.17)
plt.xlim(0.025,0.8)
Eend=80
linestyles=['--','-.','-.',':','-','-','-.','--','-.','--']
markers=[None,'x','d','v',None,'P',None,'*',9,'o']
colors=['orange','g','g','orange','red','brown','c','m','blue','y']
for i in range(0,len(S_coef_selected)):
    trendpoly = np.poly1d(S_coef_selected[i])  
    plt.plot(E[0:Eend],S_coef_selected[i][0:Eend]/summed_S_coef[0:Eend],linewidth=2,markersize=8,\
             label=Label_selected[i],linestyle=linestyles[i],marker=markers[i]\
             ,color=colors[i],markevery=5
            )
handles, labels = plt.gca().get_legend_handles_labels()
temp=plt.plot([0,0],[1,1])
handles.append(temp[0])
handles[-1].set(color='red',linestyle='-',marker='v')
labels.append("0 keV")
order = [0,1,2,3,5,4,9,6,7,8,10]
# order = [0,1,2,7,3,5,4,8,9,6]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],bbox_to_anchor=(1, 1), fontsize=19,ncol=1,loc='upper left') 

# plt.legend()
plt.grid(which='major', axis='y',linestyle='--')
# plt.legend(ncol=2)

In [ ]:
print(np.poly1d(summed_S_coef)[0]*1000,np.poly1d(summed_S_coef)[1],np.poly1d(summed_S_coef)[2]*2/1000)

In [ ]:
x=[0,100,200,300,400,500,600,700,800,900,1000]
# y=[0.783,0.773,0.763,0.757,0.747,0.743,0.733,0.723,0.717,0.71,0.703] #l=1
y=[0.437,0.437,0.437,0.437,0.447,0.447,0.447,0.453,0.457,0.457,0.457] #l=2,3
plt.plot(x,y)
trend = np.polyfit(x,y,2)
trendpoly = np.poly1d(trend) 
x_new=np.arange(0,2738,100)
# x_new=np.arange(0,4671,100)
plt.plot(x_new,trendpoly(x_new),color='b',label='fit-quad')
trendpoly(4671)

In [ ]:
x=[6.16,0.40,2.39,1.08,0.88,1.12,1.12,0.45,2.61,0.80,0.88,0.99,0.79,0.81,1.15,0.28,0.73,2.26]
x=np.log(x)
plt.hist(x, bins=[-2.1,-1.5,-0.9,-0.3,0.3,0.9,1.5,2.1],alpha=0.5, histtype='bar', ec='black')  # density=False would make counts
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Number',fontsize=15)
plt.xlabel('ln(Gp_strip/Gp_res)',fontsize=15);
yint = []
locs, labels = plt.yticks()
for each in locs:
    yint.append(int(each))
plt.yticks(yint)
plt.xticks(np.arange(-1.8, 2.1, 0.6))
x_mine=x
plt.show()

In [ ]:
x=[1.89288846397085,0.899349046346424,-0.037968925537849,0.048790164169432,-0.849599357375483,-0.286264637770808,-0.140581950621189,0.496436886313891,1.72923911224672,0.622942921886697,0.54654370636807,-1.145132304303,0.23638877806423,-0.664976303593249,-0.769133087537867,0.007117467768864,-0.059375402416003,-0.380772495517793,0.323128821447529,-0.08701137698963,0.95467497488247,-0.08701137698963,0.597003320007043,0.451985123743057,-0.446498273572082,-1.84568781125361,0.750128811097879,0.221374545685528,-0.450833713138015,2.43101088158287,0.865511924998246,0.098467246357092,1.34992671694902,0.572519192771331,0.2559702398171,-0.503445516468368,-0.219244910898552,0.852777326151829,0.122217632724249,1.19383913166669,0.883767540168595,0.631271776841858,-0.122602322092332,0.599118230911669,-0.084557388028063,-0.07333127308555,1.26633504604091,0.62057648772511,0.643956936369174,1.11185751541813,0.364643113587909,0.217589695631412,-0.188794071299572,-0.289861316027392,0.108854404912082,0.158541028128551,-0.113983445069946,0.181321056460371,0.058321610434984,-1.37731353315633,-0.451519517868583,-0.482562158493131,0.107502220569687,0.192684343829501,-0.031048482693275,0.721345834918624,0.465373360861573,0.937504128273353,0.444685821261446,0.056050898414186,0.65232518603969,0.121285666207586,0.188966099512623,-0.023268626939354,-0.2484613592985,0.207014169384326]
plt.hist(x, bins=[-2.1,-1.5,-0.9,-0.3,0.3,0.9,1.5,2.1],alpha=0.5, histtype='bar', ec='black')  # density=False would make counts
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Number',fontsize=15)
plt.xlabel('ln(Gp_calc/Gp_exp)',fontsize=15);
yint = []
locs, labels = plt.yticks()
for each in locs:
    yint.append(int(each))
plt.yticks(yint)
plt.xticks(np.arange(-1.8, 2.1, 0.6))
x_orig=x
plt.show()

In [ ]:
x=[1.12,1.12,0.45,2.61,0.80,1.31,0.99,0.79,0.81,1.15,0.28,0.73,2.26]
x=np.log(x)
x=np.append(x,[1.89288846397085,0.899349046346424,-0.037968925537849,0.048790164169432,-0.849599357375483,-0.286264637770808,-0.140581950621189,0.496436886313891,1.72923911224672,0.622942921886697,0.54654370636807,-1.145132304303,0.23638877806423,-0.664976303593249,-0.769133087537867,0.007117467768864,-0.059375402416003,-0.380772495517793,0.323128821447529,-0.08701137698963,0.95467497488247,-0.08701137698963,0.597003320007043,0.451985123743057,-0.446498273572082,-1.84568781125361,0.750128811097879,0.221374545685528,-0.450833713138015,2.43101088158287,0.865511924998246,0.098467246357092,1.34992671694902,0.572519192771331,0.2559702398171,-0.503445516468368,-0.219244910898552,0.852777326151829,0.122217632724249,1.19383913166669,0.883767540168595,0.631271776841858,-0.122602322092332,0.599118230911669,-0.084557388028063,-0.07333127308555,1.26633504604091,0.62057648772511,0.643956936369174,1.11185751541813,0.364643113587909,0.217589695631412,-0.188794071299572,-0.289861316027392,0.108854404912082,0.158541028128551,-0.113983445069946,0.181321056460371,0.058321610434984,-1.37731353315633,-0.451519517868583,-0.482562158493131,0.107502220569687,0.192684343829501,-0.031048482693275,0.721345834918624,0.465373360861573,0.937504128273353,0.444685821261446,0.056050898414186,0.65232518603969,0.121285666207586,0.188966099512623,-0.023268626939354,-0.2484613592985,0.207014169384326]
)
plt.hist(x, bins=[-2.1,-1.5,-0.9,-0.3,0.3,0.9,1.5,2.1],alpha=0.5, histtype='bar', ec='black')  # density=False would make counts
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Number',fontsize=15)
plt.xlabel('ln(Gp_calc/Gp_exp)',fontsize=15);
yint = []
locs, labels = plt.yticks()
for each in locs:
    yint.append(int(each))
plt.yticks(yint)
plt.xticks(np.arange(-1.8, 2.1, 0.6))
x_new=x
plt.show()

In [ ]:
import scipy
data=x_mine
bins=[-2.1,-1.5,-0.9,-0.3,0.3,0.9,1.5,2.1]
_, bins, _ = plt.hist(data, bins=bins, density=0, alpha=0.5,histtype='bar', ec='black')
mu, sigma = scipy.stats.norm.fit(data)

best_fit_line = scipy.stats.norm.pdf(bins, mu, sigma)

plt.plot(bins, best_fit_line*10)

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Number',fontsize=15)
plt.xlabel('ln(Gp_calc/Gp_exp)',fontsize=15);

plt.xticks(np.arange(-1.8, 2.1, 0.6))
plt.show()
mu,sigma

In [ ]:
(0.03720481602579003/np.sqrt(2), 0.7099829024501909/np.sqrt(2))

In [ ]:
(0.2236879430379645/np.sqrt(2), 0.6846843262687741/np.sqrt(2))